In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"#让所有不是在最后一行的单个变量也能print
import warnings
warnings.filterwarnings("ignore")

# 随机森林与Adaboost与XGBoost

## pandasprofilling

In [32]:
import pandas as pd
merge_loan = pd.read_csv("merge_loan_test.csv")
merge_loan_train = pd.read_csv("merge_loan.csv")

In [ ]:
import numpy as np
import pandas_profiling

#To display the report in a Jupyter notebook, run:

merge_loan.profile_report(style={'full_width':True})
#To retrieve the list of variables which are rejected due to high correlation:

#profile = df.profile_report()

## 热力图

In [ ]:
feature_group1=merge_loan[['I_1', 'I_2', 'I_3', 'P_1', 'P_2', 'P_3','log_sum_send_amount','lend_period','view_no','30days_overdue_times','is_30days_overdue']]

In [ ]:
import seaborn as sns
sns.heatmap(feature_group1.corr(),annot=True)

In [ ]:
feature_group2=merge_loan[['sum_his_lend_amount', 'mean_his_overdue_times', 'sum_before_30_trans_amount', 'mean_every_day_trans_amount', 'cv_mean_every_day_trans_amount', 'sum_every_trans_amount','on_sale_rate','sum_before_30_trans_count','sum_every_trans_count','nearest_trans_day','mean_every_trans_trans_amount','is_30days_overdue']]

In [ ]:
sns.heatmap(feature_group2.corr(),annot=True)

In [ ]:
#可以看到按相关性相对高的特征结合 特征间相关系数都特别低 说明特征离散得不错

In [ ]:
feature_group3=merge_loan[['bankcard_credit_count','bankcard_debit_count','wallet_weixin_count','wallet_alipay_count','alipay_huabei_count','is_30days_overdue']]

In [ ]:
sns.heatmap(feature_group3.corr(),annot=True)

In [ ]:
feature_group4=merge_loan[['mean_good_status','store_num','terminal_num','on_sale_or_not','mean_payer_uid','is_30days_overdue']]

In [ ]:
sns.heatmap(feature_group4.corr(),annot=True)

In [ ]:
del merge_loan

## 层次聚类图

In [ ]:
sns.clustermap(feature_group4)

### 导入需要的库
### 这边暂时先不卡方分箱，离群检测等，现测试一波看下准确率

In [ ]:
# k 折交叉验证（k-fold cross validation
import pandas as pd
import numpy as np
import matplotlib as plt
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import cross_val_score


## 选择需要的feature

In [33]:
feature=merge_loan[['I_1', 'I_2', 'I_3', 'P_1', 'P_2', 'P_3','log_sum_send_amount','lend_period','view_no',\
                    'sum_his_lend_amount', 'mean_his_overdue_times', 'sum_before_30_trans_amount', 'mean_every_day_trans_amount', 'cv_mean_every_day_trans_amount', 'sum_every_trans_amount','on_sale_rate','sum_before_30_trans_count','sum_every_trans_count','nearest_trans_day','mean_every_trans_trans_amount',\
                    'mean_good_status','store_num','terminal_num','on_sale_or_not','mean_payer_uid',\
                    'bankcard_credit_count','bankcard_debit_count','wallet_weixin_count','wallet_alipay_count','alipay_huabei_count']]

In [34]:
feature_train=merge_loan_train[['I_1', 'I_2', 'I_3', 'P_1', 'P_2', 'P_3','log_sum_send_amount','lend_period','view_no',\
                    'sum_his_lend_amount', 'mean_his_overdue_times', 'sum_before_30_trans_amount', 'mean_every_day_trans_amount', 'cv_mean_every_day_trans_amount', 'sum_every_trans_amount','on_sale_rate','sum_before_30_trans_count','sum_every_trans_count','nearest_trans_day','mean_every_trans_trans_amount',\
                    'mean_good_status','store_num','terminal_num','on_sale_or_not','mean_payer_uid',\
                    'bankcard_credit_count','bankcard_debit_count','wallet_weixin_count','wallet_alipay_count','alipay_huabei_count','is_30days_overdue']]

In [35]:
feature['cv_mean_every_day_trans_amount']=feature['cv_mean_every_day_trans_amount'].fillna(0)

In [36]:
feature_train['cv_mean_every_day_trans_amount']=feature_train['cv_mean_every_day_trans_amount'].fillna(0)

In [37]:
feature=feature.drop(['bankcard_credit_count','bankcard_debit_count','wallet_weixin_count','wallet_alipay_count','alipay_huabei_count'],axis=1)

In [38]:
feature_train=feature_train.drop(['bankcard_credit_count','bankcard_debit_count','wallet_weixin_count','wallet_alipay_count','alipay_huabei_count'],axis=1)

In [39]:
feature=feature.drop(['I_1', 'I_2', 'I_3', 'P_1', 'P_2', 'P_3'],axis=1)

In [40]:
feature_train=feature_train.drop(['I_1', 'I_2', 'I_3', 'P_1', 'P_2', 'P_3'],axis=1)

In [41]:
import numpy as np
feature['sum_his_lend_amount']=feature['sum_his_lend_amount'].map(lambda i:np.log(i) if i>0 else 0)
feature_train['sum_his_lend_amount']=feature_train['sum_his_lend_amount'].map(lambda i:np.log(i) if i>0 else 0)

In [46]:
feature['log_sum_send_amount']=feature['log_sum_send_amount'].map(lambda i:np.exp(i) if i>0 else 0)

In [61]:
feature=feature.apply(lambda x: (x - np.min(x)) / (np.max(x) - np.min(x)))
feature_train=feature_train.apply(lambda x: (x - np.min(x)) / (np.max(x) - np.min(x)))

In [ ]:
feature.columns

In [ ]:
feature_train.columns

In [62]:
feature.head(20)

,log_sum_send_amount,lend_period,view_no,sum_his_lend_amount,mean_his_overdue_times,sum_before_30_trans_amount,mean_every_day_trans_amount,cv_mean_every_day_trans_amount,sum_every_trans_amount,on_sale_rate,sum_before_30_trans_count,sum_every_trans_count,nearest_trans_day,mean_every_trans_trans_amount,mean_good_status,store_num,terminal_num,on_sale_or_not,mean_payer_uid
0,0.207351,1.000000,0.283476,0.585836,0.342857,0.708267,0.681788,0.116594,0.762552,0.989301,0.189189,0.626667,0.000000,0.561611,0.935056,0.000000,0.024390,0.105426,0.473908
1,0.434588,0.454545,0.082911,0.567290,0.388000,0.757589,0.712411,0.372269,0.784500,0.996128,0.310811,0.773333,0.000000,0.744346,0.818146,0.000000,0.000000,0.116402,0.088984
2,0.303764,0.181818,0.348594,0.567290,0.388000,0.548158,0.764678,0.320512,0.711933,0.997082,0.040541,0.160000,0.000000,0.794644,0.449864,0.000000,0.024390,0.146341,0.361470
3,0.738352,1.000000,0.141738,0.585836,0.342857,0.838826,0.805407,0.332435,0.856913,0.998964,0.797297,0.973333,0.000000,0.784040,0.932021,0.000000,0.024390,0.126582,0.821560
4,0.303764,0.181818,0.082911,0.602248,0.229310,0.677505,0.731782,0.172372,0.738287,0.991337,0.040541,0.226667,0.144928,0.565122,0.904829,0.000000,0.000000,0.156962,0.481249
5,0.691458,1.000000,0.685003,0.567290,0.388000,0.737560,0.694447,0.271281,0.772912,0.969885,0.891892,1.000000,0.000000,0.610681,0.891572,0.000000,0.024390,0.114014,0.420009
6,0.303764,0.727273,0.490332,0.585836,0.342857,0.488854,0.594147,0.000000,0.595828,1.000000,0.000000,0.026667,0.115942,0.770418,0.328704,0.000000,0.048780,0.000000,0.496544
7,0.738352,0.272727,0.612580,0.738352,0.000000,0.844267,0.792070,0.137174,0.855037,0.986872,0.689189,1.000000,0.000000,0.585341,0.984555,0.000000,0.000000,0.172561,0.446453
8,0.434588,1.000000,0.307560,0.207351,0.200000,0.756071,0.791299,0.201353,0.792487,0.999581,0.135135,0.226667,0.014493,0.902783,0.688172,0.000000,0.000000,0.064516,0.480303
9,0.303764,0.454545,0.412016,0.392472,0.800000,0.805717,0.845970,0.267214,0.818018,1.000000,0.108108,0.133333,0.202899,0.967967,0.309524,0.000000,0.000000,0.000000,0.208855


In [27]:
feature['mean_every_day_trans_amount'][feature['mean_every_day_trans_amount']==0]

362    0.0
Name: mean_every_day_trans_amount, dtype: float64

In [28]:
feature_train['mean_every_day_trans_amount'][feature_train['mean_every_day_trans_amount']==0]

54      0.0
235     0.0
355     0.0
555     0.0
1016    0.0
Name: mean_every_day_trans_amount, dtype: float64

可以看到日均交易额变异系数明显有缺失值 分析是除以均值时均值为0所致

# 最终结果

## 随机森林

In [72]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score
train_data = feature_train.drop(['is_30days_overdue'], axis=1)
train_target = feature_train['is_30days_overdue']
# X_train, X_test, y_train, y_test = train_test_split(train_data,train_target,test_size=0.25, random_state=1)
# clf = RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
#             max_depth=7, max_features=15, max_leaf_nodes=None,
#             min_impurity_decrease=0.0, min_impurity_split=None,
#             min_samples_leaf=2, min_samples_split=4,
#             min_weight_fraction_leaf=0.0, n_estimators=70, n_jobs=1,
#             oob_score=False, random_state=None, verbose=0,
#             warm_start=False)
from sklearn.externals import joblib


# clf = joblib.load('RFC.m')
# # clf = clf.fit(train_data,train_target)
# print(clf)
# y_RF = clf.predict(feature)
# clf = joblib.load('RFC2.m')
# y_RF2 = clf.predict(feature)


# f1_score(y_test,y_pridict)
clf = joblib.load('RF_gcy_872209.m')
clf
y_RF2 = clf.predict(feature)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=7, max_features=15, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=2, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=70, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [73]:
y_RF2

array([1., 0., 1., 0., 1., 0., 1., 0., 1., 1., 0., 0., 1., 1., 0., 1., 0.,
       0., 1., 0., 1., 1., 1., 1., 0., 1., 0., 1., 0., 0., 1., 0., 0., 1.,
       0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 1., 1., 0., 1., 0., 0., 1.,
       1., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 0., 0., 1., 0., 0., 1.,
       1., 0., 0., 0., 0., 1., 0., 1., 1., 0., 1., 0., 1., 0., 0., 1., 1.,
       0., 0., 0., 0., 0., 1., 1., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0.,
       1., 0., 1., 0., 1., 1., 0., 1., 0., 1., 0., 0., 1., 0., 1., 0., 0.,
       1., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 0., 0., 0., 1., 0., 1.,
       1., 1., 1., 0., 0., 0., 0., 0., 0., 1., 1., 1., 0., 0., 1., 0., 0.,
       1., 0., 0., 0., 0., 1., 1., 1., 0., 1., 0., 0., 0., 0., 0., 1., 1.,
       0., 0., 0., 1., 1., 1., 0., 0., 0., 0., 1., 1., 0., 0., 1., 1., 1.,
       1., 0., 0., 1., 0., 0., 1., 1., 1., 0., 0., 1., 0., 1., 0., 1., 0.,
       1., 1., 1., 1., 1., 1., 0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 0.,
       1., 1., 0., 0., 0.

In [74]:
str(y_RF2).count('0')
str(y_RF2).count('1')

241

212

In [75]:
import pandas as pd
import numpy as np
df_RF2 = pd.DataFrame()
print(df_RF2)

Empty DataFrame
Columns: []
Index: []


In [76]:
df_RF2.insert(0, 'y_predict', y_RF2)
df_RF2.insert(0, 'apply_id', merge_loan['apply_id'].tolist())

In [77]:
df_RF2.head()

,apply_id,y_predict
0,50557025,1.0
1,49510913,0.0
2,50782824,1.0
3,51967846,0.0
4,51105993,1.0


In [78]:
df_RF2.to_csv('RFpredict_changef1_11252210.csv')

## 决策树

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score
train_data = feature_train.drop(['is_30days_overdue'], axis=1)
train_target = feature_train['is_30days_overdue']
# X_train, X_test, y_train, y_test = train_test_split(train_data,train_target,test_size=0.25, random_state=1)
from sklearn.externals import joblib
clf = joblib.load('DTC_changef1_66.m')

# clf = clf.fit(train_data,train_target)
print(clf)
y_DT = clf.predict(feature)
# f1_score(y_test,y_pridict)

In [ ]:
y_DT

array([1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0,
       0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0,
       1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1,
       0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0,
       1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1,
       1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1,
       1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0], dtype=int64)

In [ ]:
str(y_DT).count('0')
str(y_DT).count('1')

In [ ]:
before_=np.array([1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0])

In [ ]:
str(before_).count('0')
str(before_).count('1')

In [ ]:
import pandas as pd
import numpy as np
df_DT = pd.DataFrame()
print(df_DT)
df_DT.insert(0, 'y_predict', y_DT)
df_DT.insert(0, 'apply_id', merge_loan['apply_id'].tolist())
df_DT.head()

In [ ]:
df_DT.head()

In [ ]:
df_DT.to_csv('DT_predict_changef1_1125.csv')

## Adaboost

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score
train_data = feature_train.drop(['is_30days_overdue'], axis=1)
train_target = feature_train['is_30days_overdue']
# X_train, X_test, y_train, y_test = train_test_split(train_data,train_target,test_size=0.25, random_state=1)
# clf = AdaBoostClassifier(algorithm='SAMME.R',
#           base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=5,
#             max_features=15, max_leaf_nodes=None,
#             min_impurity_decrease=0.0, min_impurity_split=None,
#             min_samples_leaf=2, min_samples_split=12,
#             min_weight_fraction_leaf=0.0, presort=False, random_state=None,
#             splitter='random'),
#           learning_rate=1.0, n_estimators=1, random_state=None)
from sklearn.externals import joblib
clf = joblib.load('ABC_changef1_66.m')
# clf = clf.fit(train_data,train_target)
print(clf)
y_AB = clf.predict(feature.values)
# f1_score(y_test,y_pridict)

In [ ]:
y_AB

In [ ]:
str(y_AB).count('0')
str(y_AB).count('1')

In [ ]:
import pandas as pd
import numpy as np
df_AB = pd.DataFrame()
print(df_AB)
df_AB.insert(0, 'y_predict', y_AB)
df_AB.insert(0, 'apply_id', merge_loan['apply_id'].tolist())
df_AB.head()

In [ ]:
df_AB.to_csv('ABC_predict_changef1_1125.csv')

## XGBoost

In [66]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score
from xgboost import XGBClassifier
import xgboost as xgb
train_data = feature_train.drop(['is_30days_overdue'], axis=1)
train_target = feature_train['is_30days_overdue']
# X_train, X_test, y_train, y_test = train_test_split(train_data,train_target,test_size=0.25, random_state=1)
# clf = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
#        colsample_bynode=1, colsample_bytree=0.4, gamma=0,
#        learning_rate=0.1, max_delta_step=0, max_depth=2,
#        min_child_weight=1, missing=None, n_estimators=200, n_jobs=1,
#        nthread=None, objective='binary:logistic', random_state=0,
#        reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
#        silent=None, subsample=1, verbosity=1)
from sklearn.externals import joblib
# clf = joblib.load('XGB_changef1_62.m')
clf = joblib.load('XGB_gcy_872149.m')
# clf = clf.fit(train_data,train_target)
print(clf)
feature2=feature.copy()
feature2.columns=['f11','f10','f0','f8','f5','f2','f7','f4','f12','f6','f13','f17','f9','f3','f14','f1','f18','f16','f15']
feature2[['f0','f1','f2','f3','f4','f5','f6','f7','f8','f9','f10','f11','f12','f13','f14','f15','f16','f17','f18']]
feature
# y_XGB = clf.predict(feature2[['f0','f1','f2','f3','f4','f5','f6','f7','f8','f9','f10','f11','f12','f13','f14','f15','f16','f17','f18']])

dtest = feature.values
y_XGB=clf.predict(dtest)
del feature2
# f1_score(y_test,y_pridict)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=0.5, gamma=0,
       learning_rate=0.1, max_delta_step=0, max_depth=4,
       min_child_weight=1, missing=nan, n_estimators=250, n_jobs=1,
       nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=None, subsample=1, verbosity=1)


,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18
0,0.283476,0.000000,0.708267,0.561611,0.116594,0.342857,0.989301,0.681788,0.585836,0.000000,1.000000,0.207351,0.762552,0.189189,0.935056,0.473908,0.105426,0.626667,0.024390
1,0.082911,0.000000,0.757589,0.744346,0.372269,0.388000,0.996128,0.712411,0.567290,0.000000,0.454545,0.434588,0.784500,0.310811,0.818146,0.088984,0.116402,0.773333,0.000000
2,0.348594,0.000000,0.548158,0.794644,0.320512,0.388000,0.997082,0.764678,0.567290,0.000000,0.181818,0.303764,0.711933,0.040541,0.449864,0.361470,0.146341,0.160000,0.024390
3,0.141738,0.000000,0.838826,0.784040,0.332435,0.342857,0.998964,0.805407,0.585836,0.000000,1.000000,0.738352,0.856913,0.797297,0.932021,0.821560,0.126582,0.973333,0.024390
4,0.082911,0.000000,0.677505,0.565122,0.172372,0.229310,0.991337,0.731782,0.602248,0.144928,0.181818,0.303764,0.738287,0.040541,0.904829,0.481249,0.156962,0.226667,0.000000
5,0.685003,0.000000,0.737560,0.610681,0.271281,0.388000,0.969885,0.694447,0.567290,0.000000,1.000000,0.691458,0.772912,0.891892,0.891572,0.420009,0.114014,1.000000,0.024390
6,0.490332,0.000000,0.488854,0.770418,0.000000,0.342857,1.000000,0.594147,0.585836,0.115942,0.727273,0.303764,0.595828,0.000000,0.328704,0.496544,0.000000,0.026667,0.048780
7,0.612580,0.000000,0.844267,0.585341,0.137174,0.000000,0.986872,0.792070,0.738352,0.000000,0.272727,0.738352,0.855037,0.689189,0.984555,0.446453,0.172561,1.000000,0.000000
8,0.307560,0.000000,0.756071,0.902783,0.201353,0.200000,0.999581,0.791299,0.207351,0.014493,1.000000,0.434588,0.792487,0.135135,0.688172,0.480303,0.064516,0.226667,0.000000
9,0.412016,0.000000,0.805717,0.967967,0.267214,0.800000,1.000000,0.845970,0.392472,0.202899,0.454545,0.303764,0.818018,0.108108,0.309524,0.208855,0.000000,0.133333,0.000000


,log_sum_send_amount,lend_period,view_no,sum_his_lend_amount,mean_his_overdue_times,sum_before_30_trans_amount,mean_every_day_trans_amount,cv_mean_every_day_trans_amount,sum_every_trans_amount,on_sale_rate,sum_before_30_trans_count,sum_every_trans_count,nearest_trans_day,mean_every_trans_trans_amount,mean_good_status,store_num,terminal_num,on_sale_or_not,mean_payer_uid
0,0.207351,1.000000,0.283476,0.585836,0.342857,0.708267,0.681788,0.116594,0.762552,0.989301,0.189189,0.626667,0.000000,0.561611,0.935056,0.000000,0.024390,0.105426,0.473908
1,0.434588,0.454545,0.082911,0.567290,0.388000,0.757589,0.712411,0.372269,0.784500,0.996128,0.310811,0.773333,0.000000,0.744346,0.818146,0.000000,0.000000,0.116402,0.088984
2,0.303764,0.181818,0.348594,0.567290,0.388000,0.548158,0.764678,0.320512,0.711933,0.997082,0.040541,0.160000,0.000000,0.794644,0.449864,0.000000,0.024390,0.146341,0.361470
3,0.738352,1.000000,0.141738,0.585836,0.342857,0.838826,0.805407,0.332435,0.856913,0.998964,0.797297,0.973333,0.000000,0.784040,0.932021,0.000000,0.024390,0.126582,0.821560
4,0.303764,0.181818,0.082911,0.602248,0.229310,0.677505,0.731782,0.172372,0.738287,0.991337,0.040541,0.226667,0.144928,0.565122,0.904829,0.000000,0.000000,0.156962,0.481249
5,0.691458,1.000000,0.685003,0.567290,0.388000,0.737560,0.694447,0.271281,0.772912,0.969885,0.891892,1.000000,0.000000,0.610681,0.891572,0.000000,0.024390,0.114014,0.420009
6,0.303764,0.727273,0.490332,0.585836,0.342857,0.488854,0.594147,0.000000,0.595828,1.000000,0.000000,0.026667,0.115942,0.770418,0.328704,0.000000,0.048780,0.000000,0.496544
7,0.738352,0.272727,0.612580,0.738352,0.000000,0.844267,0.792070,0.137174,0.855037,0.986872,0.689189,1.000000,0.000000,0.585341,0.984555,0.000000,0.000000,0.172561,0.446453
8,0.434588,1.000000,0.307560,0.207351,0.200000,0.756071,0.791299,0.201353,0.792487,0.999581,0.135135,0.226667,0.014493,0.902783,0.688172,0.000000,0.000000,0.064516,0.480303
9,0.303764,0.454545,0.412016,0.392472,0.800000,0.805717,0.845970,0.267214,0.818018,1.000000,0.108108,0.133333,0.202899,0.967967,0.309524,0.000000,0.000000,0.000000,0.208855


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [67]:
y_XGB

array([0., 0., 1., 0., 0., 0., 1., 0., 1., 1., 0., 0., 1., 1., 0., 1., 0.,
       0., 1., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0.,
       0., 1., 0., 0., 0., 1., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 1.,
       1., 0., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 1., 0., 0., 1., 1.,
       0., 0., 0., 0., 0., 1., 1., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0.,
       1., 0., 1., 0., 1., 1., 0., 1., 0., 1., 0., 0., 1., 0., 1., 0., 0.,
       1., 0., 0., 0., 0., 0., 1., 0., 1., 1., 0., 0., 0., 0., 1., 0., 1.,
       1., 1., 1., 0., 1., 0., 0., 0., 0., 1., 1., 1., 0., 0., 1., 0., 0.,
       1., 0., 0., 0., 0., 1., 0., 1., 0., 1., 0., 0., 0., 0., 0., 1., 0.,
       0., 0., 0., 1., 1., 1., 0., 0., 0., 0., 1., 1., 0., 0., 1., 1., 0.,
       1., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 1., 0., 1., 0., 1., 0.,
       0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
       1., 0., 0., 1., 0.

In [68]:
str(y_XGB).count('0')
str(y_XGB).count('1')

287

166

In [69]:
import pandas as pd
import numpy as np
df_XGB = pd.DataFrame()
print(df_XGB)
df_XGB.insert(0, 'y_predict', y_XGB)
df_XGB.insert(0, 'apply_id', merge_loan['apply_id'].tolist())
df_XGB.head()

Empty DataFrame
Columns: []
Index: []


,apply_id,y_predict
0,50557025,0.0
1,49510913,0.0
2,50782824,1.0
3,51967846,0.0
4,51105993,0.0


In [70]:
df_XGB.head(10)

,apply_id,y_predict
0,50557025,0.0
1,49510913,0.0
2,50782824,1.0
3,51967846,0.0
4,51105993,0.0
5,52709949,0.0
6,49717847,1.0
7,51179153,0.0
8,50083070,1.0
9,51727931,1.0


In [71]:
df_XGB.to_csv('XGB_predict_changef1_12152153.csv')

# 下面不管

## 用train_test_split先试一下

用f1非k折的单个决策树还有0.63

In [ ]:
y = feature['is_30days_overdue']##标签
X = feature.drop(['is_30days_overdue'], axis=1).values##特征

### DecisionTree

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score
train_data = feature.drop(['is_30days_overdue'], axis=1)
train_target = y
X_train, X_test, y_train, y_test = train_test_split(train_data,train_target,test_size=0.25, random_state=1)
clf = DecisionTreeClassifier(max_depth=4,criterion='gini')
clf = clf.fit(X_train,y_train)
print(clf)
y_pridict = clf.predict(X_test)
f1_score(y_test,y_pridict)

#### AUC分数

In [ ]:
from sklearn.metrics import roc_auc_score
roc=roc_auc_score(y_test,y_pridict)
print("The AUROC=%f",roc)

True Positive（TP）意思表示做出Positive的判定，而且判定是正确的。  
因此，TP的数值表示正确的Positive判定的个数。  
同理，False Positive（TP）数值表示错误的Positive判定的个数。  
依此，True Negative（TN）数值表示正确的Negative判定个数。   
False Negative（FN）数值表示错误的Negative判定个数。  
 
Precision、Recall、Accuracy、F1 Score（F Score） 四个概念定义：  
precision = TP / (TP + FP)   
recall = TP / (TP + FN)   
accuracy = (TP + TN) / (TP + FP + TN + FN)   
F1 Score = P*R/2(P+R)，其中P和R分别为 precision 和 recall 如果某个二元分类问题， 
训练拟合得到了几个模型假设，那么通常我们选择在验证集上，F1 Score 数值最大的那个模型假设。  

下图中  
TN FN  
FP TP  

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import seaborn as sns
# y_true = [2, 1, 0, 1, 2, 0]
# y_pred = [2, 0, 0, 1, 2, 1]
print(classification_report(y_test,y_pridict))
C=confusion_matrix(y_test,y_pridict)
print(C, end='\n\n')
df_cm = pd.DataFrame(C)
sns.heatmap(df_cm,annot=True)
# y_true = ["cat", "ant", "cat", "cat", "ant", "bird"]
# y_pred = ["ant", "ant", "cat", "cat", "ant", "cat"]
# C2 = confusion_matrix(y_true, y_pred, labels=["ant", "bird", "cat"])
# print(C2)


模型训练时没有偷懒，没有在较少的label上准确率低，所以直接用交叉验证不用train_test_split看效果更好一些

可以看到1被预测成1/1的总数为0.73 比真实准确率0.71高 说明预测的不错

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier
train_data = feature.drop(['is_30days_overdue'], axis=1)
train_target = y
X_train, X_test, y_train, y_test = train_test_split(train_data,train_target,test_size=0.25, random_state=1)
clf = RandomForestClassifier(max_depth=4,criterion='gini')
clf = clf.fit(X_train,y_train)
print(clf)
y_pridict = clf.predict(X_test)
f1_score(y_test,y_pridict)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
train_data = feature.drop(['is_30days_overdue'], axis=1)
train_target = y
X_train, X_test, y_train, y_test = train_test_split(train_data,train_target,test_size=0.25, random_state=1)
clf = DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=6,
            max_features=10, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=5, min_samples_split=10,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')
clf = clf.fit(X_train,y_train)
print(clf)
y_pridict = clf.predict(X_test)
f1_score(y_test,y_pridict)

In [ ]:
y_train

In [ ]:
# DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=6,
#             max_features=10, max_leaf_nodes=None,
#             min_impurity_decrease=0.0, min_impurity_split=None,
#             min_samples_leaf=5, min_samples_split=10,
#             min_weight_fraction_leaf=0.0, presort=False, random_state=None,
#             splitter='best')

## 交叉验证测试
 cross_val_score函数直接整合了StratifiedKFold的默认函数~

In [ ]:

from sklearn.metrics import f1_score
from xgboost import XGBClassifier
y = feature['is_30days_overdue']##标签
X = feature.drop(['is_30days_overdue'], axis=1).values##特征

classifiers = [DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=4,
            max_features=5, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=4, min_samples_split=14,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
               XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=0.4, gamma=0,
       learning_rate=0.1, max_delta_step=0, max_depth=2,
       min_child_weight=1, missing=None, n_estimators=200, n_jobs=1,
       nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=None, subsample=1, verbosity=1),
               AdaBoostClassifier(algorithm='SAMME.R',
          base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=5,
            max_features=15, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=2, min_samples_split=12,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='random'),
          learning_rate=1.0, n_estimators=1, random_state=None),
              ]
#RandomForestClassifier DesitionTreeClassifier 3个Adaboost XGBoost
data = {'DesitionTree':[],'XGBClassifier':[]}
result = pd.DataFrame(data)
for i in range(15):
    list=[]
    for clf in classifiers:
        score = cross_val_score(clf, X, y, cv=10, scoring='f1')#cv=10：10 折交叉验证法，scoring='accuracy'：返回测试精度
        #因为不逾期：逾期约7:5,所以用accuracy至少会有57.2814%正确率，所以用f1进行scoring
        #print([np.mean(score)])#显示测试精度平均值
        list.append(np.mean(score))
    result.loc[i]=list
result
    #print(f1_score(X,y))
#先adaboost再randomforest adaboost结果还好一些

### DecisionTree,AdaBoost,RandomForest

In [ ]:
from sklearn.metrics import f1_score
y = feature['is_30days_overdue']##标签
X = feature.drop(['is_30days_overdue'], axis=1).values##特征

classifiers = [DecisionTreeClassifier(max_depth=4,criterion='gini'),AdaBoostClassifier(
    random_state=2), RandomForestClassifier(random_state=2),
               DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=4, max_features=10, max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, min_samples_leaf=3, min_samples_split=5, min_weight_fraction_leaf=0.0, presort=False, random_state=None, splitter='best'),
             DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3, max_features=10, max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, min_samples_leaf=3, min_samples_split=12, min_weight_fraction_leaf=0.0, presort=False, random_state=None, splitter='best') ]
#adaboot和第四个decisiontree明显提升
for clf in classifiers:
    score = cross_val_score(clf, X, y, cv=10, scoring='f1')#cv=10：10 折交叉验证法，scoring='accuracy'：返回测试精度
    #因为不逾期：逾期约7:5,所以用accuracy至少会有57.2814%正确率，所以用f1进行scoring
    print([np.mean(score)])#显示测试精度平均值
    #print(f1_score(X,y))
#先adaboost再randomforest adaboost结果还好一些

这次（去I1I2I3P1P2P3）是Adaboost降了 DecisionTree升了 

### 多次Adaboost

In [ ]:
from sklearn.metrics import f1_score
from xgboost import XGBClassifier
y = feature['is_30days_overdue']##标签
X = feature.drop(['is_30days_overdue'], axis=1).values##特征

classifiers = [RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=7, max_features=15, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=2, min_samples_split=4,
            min_weight_fraction_leaf=0.0, n_estimators=70, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),\
               DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=4,
            max_features=5, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=4, min_samples_split=14,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),\
        AdaBoostClassifier(algorithm='SAMME.R',
          base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=5,
            max_features=15, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=2, min_samples_split=12,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='random'),
          learning_rate=1.0, n_estimators=1, random_state=None),\
              XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=0.4, gamma=0,
       learning_rate=0.1, max_delta_step=0, max_depth=2,
       min_child_weight=1, missing=None, n_estimators=200, n_jobs=1,
       nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=None, subsample=1, verbosity=1)]
#RandomForestClassifier DesitionTreeClassifier 3个Adaboost XGBoost
data = {'RandomForest':[],'DesitionTree':[],'Adaboost':[],'XGBoost':[]}
result = pd.DataFrame(data)
for i in range(7):
    list=[]
    for clf in classifiers:
        score = cross_val_score(clf, X, y, cv=10, scoring='f1')#cv=10：10 折交叉验证法，scoring='accuracy'：返回测试精度
        #因为不逾期：逾期约7:5,所以用accuracy至少会有57.2814%正确率，所以用f1进行scoring
        #print([np.mean(score)])#显示测试精度平均值
        list.append(np.mean(score))
    result.loc[i]=list
result
    #print(f1_score(X,y))
#先adaboost再randomforest adaboost结果还好一些

原来
RandomForest	DesitionTree	Adaboost1	Adaboost2	Adaboost3	XGBoost
0	0.619503	0.564263	0.532618	0.579869	0.586894	0.599188
1	0.630274	0.564539	0.590817	0.581236	0.589593	0.599188
2	0.630867	0.544616	0.594115	0.566209	0.578479	0.599188
3	0.628147	0.549137	0.563573	0.525717	0.559777	0.599188
4	0.617550	0.572193	0.573680	0.567733	0.561072	0.599188
5	0.604245	0.563753	0.601234	0.547066	0.563356	0.599188
6	0.631666	0.546882	0.490676	0.520370	0.545420	0.599188
第一次删交易方式
可以看到RandomForest收影响不大，DesitionTree疯狂提升但还是不如RF和XGB,Adaboost也有提升
	RandomForest	DesitionTree	Adaboost1	Adaboost2	Adaboost3	XGBoost
0	0.619401	0.601556	0.571297	0.556061	0.567685	0.617617
1	0.622937	0.574043	0.545617	0.554873	0.573797	0.617617
2	0.630850	0.531901	0.585603	0.579894	0.553374	0.617617
3	0.616955	0.582779	0.582336	0.550765	0.565448	0.617617
4	0.608231	0.600332	0.587368	0.506586	0.604879	0.617617
5	0.617595	0.600352	0.589645	0.556198	0.586399	0.617617
6	0.639605	0.581682	0.545618	0.583483	0.602360	0.617617
第二次删I1I2I3P1P2P3
可以看到基本维持不变，这可能是因为参数需要再调一波 所以调一下再试
RandomForest	DesitionTree	Adaboost1	Adaboost2	Adaboost3	XGBoost
0	0.630208	0.557664	0.581018	0.610085	0.589487	0.610233
1	0.613704	0.578765	0.584903	0.571300	0.542334	0.610233
2	0.634073	0.576649	0.541038	0.574370	0.610602	0.610233
3	0.622816	0.572986	0.575564	0.585786	0.599933	0.610233
4	0.617320	0.574186	0.593813	0.589618	0.600303	0.610233
5	0.619191	0.567567	0.574016	0.595418	0.591603	0.610233
6	0.624364	0.595464	0.612314	0.554814	0.608593	0.610233
最终

RandomForest	DesitionTree	Adaboost	XGBoost
0	0.616682	0.629159	0.618274	0.619243
1	0.631258	0.587805	0.608033	0.619243
2	0.619141	0.564527	0.583956	0.619243
3	0.611361	0.597486	0.565275	0.619243
4	0.640254	0.554826	0.612308	0.619243
5	0.628728	0.626035	0.546870	0.619243
6	0.630640	0.601298	0.557956	0.619243


可以看到RandomForest收影响不大，DesitionTree疯狂提升但还是不如RF和XGB,Adaboost也有提升

In [ ]:
data = {'RandomForest':[],'DesitionTree':[],'Adaboost1':[],'Adaboost2':[],'Adaboost3':[],'XGBoost':[]}
# 创建一个Series对象
result = pd.DataFrame(data)
result.iloc[0,1]=1
result

### XGBoost

In [ ]:
import xgboost as xgb#调入XGBoost模块
from xgboost import plot_importance
X = feature.drop(['is_30days_overdue'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.25, random_state=1)
feature_name = ['I_1', 'I_2', 'I_3', 'P_1', 'P_2', 'P_3','log_sum_send_amount','lend_period','view_no',\
                    'sum_his_lend_amount', 'mean_his_overdue_times', 'sum_before_30_trans_amount', 'mean_every_day_trans_amount', 'cv_mean_every_day_trans_amount', 'sum_every_trans_amount','on_sale_rate','sum_before_30_trans_count','sum_every_trans_count','nearest_trans_day','mean_every_trans_trans_amount',\
                    'mean_good_status','store_num','terminal_num','on_sale_or_not','mean_payer_uid',\
                    'bankcard_credit_count','bankcard_debit_count','wallet_weixin_count','wallet_alipay_count','alipay_huabei_count']
xgbr=xgb.XGBClassifier()#调用XGBRegressor函数‍

xgbr.fit(X_train,y_train)#拟合
fig, ax = plt.subplots(figsize=(10,8))
plot_importance(xgbr,ax=ax)

plt.show()
xgbr_y_predict=xgbr.predict(X_test)#预测
f1_score(y_test,xgbr_y_predict)

可以看到省份也完全没用 也去了 然后绕回去重跑 图记录就不存了

# 注：由这及特征图可见第三类交易类型特征不好 可以不要！
['bankcard_credit_count','bankcard_debit_count','wallet_weixin_count','wallet_alipay_count','alipay_huabei_count'

In [ ]:
X.shape[1]

In [ ]:
from sklearn.metrics import roc_auc_score
roc=roc_auc_score(y_test,xgbr_y_predict)
print("The AUROC=%f",roc)

In [ ]:
# y=y.values

#设置待选的参数        
from sklearn.tree import DecisionTreeClassifier
from sklearn.grid_search import GridSearchCV
from sklearn.model_selection import StratifiedKFold
decision_tree_classifier = DecisionTreeClassifier()
parameter_grid = {'max_depth': [2,3,4,5,6], 'max_features': [5,8,10,12,14], 'criterion': ['gini', 'entropy'],
             'min_samples_split': [5,8,10,12,14], 'min_samples_leaf': [3,4,5,7]}
cross_validation = StratifiedKFold(n_splits=10)

#将不同参数带入
gridsearch = GridSearchCV(decision_tree_classifier,
                          param_grid = parameter_grid,
                          cv = cross_validation)
gridsearch.fit(X,y)

#得分最高的参数值，并构建最佳的决策树
best_param = gridsearch.best_params_

# best_param.best_score_
# best_decision_tree_classifier = DecisionTreeClassifier(max_depth=best_param['max_depth'],
#                                                        max_features=best_param['max_features'])

In [ ]:
np.isnan(X).any()

In [ ]:
y_train.shape[0]

In [ ]:
X_train.shape[0]

In [ ]:
X_test.shape[0]

In [ ]:
for k, (train,test) in enumerate(kf):
    print(k,train,test)

In [ ]:
from sklearn.cross_validation import StratifiedKFold
import numpy as np

scores = []
kfold = StratifiedKFold(y=y_train, n_folds=10, random_state=1) # n_folds参数设置为10份
for train_index, test_index in kfold:
    DecisionTreeClassifier(max_depth=4,criterion='gini').fit(X_train[train_index], y_train[train_index])
    score = DecisionTreeClassifier(max_depth=4,criterion='gini').score(X_train[test_index], y_train[test_index])
    scores.append(score)
    print('类别分布: %s, 准确度: %.3f' % (np.bincount(y_train[train_index]), score))

# 学习曲线 调参时画

## Adaboost

In [ ]:
from sklearn.model_selection import learning_curve
import matplotlib.pyplot as plt

# 定义函数 plot_learning_curve 绘制学习曲线。
#train_sizes 初始化为 array([ 0.1  ,  0.325,  0.55 ,  0.775,  1\.   ]),
#cv 初始化为 10，以后调用函数时不再输入这两个变量

def plot_learning_curve(estimator, title, X, y, cv=10,
                        train_sizes=np.linspace(.1, 1.0, 5)):
    plt.figure(figsize=(20,10))
    plt.title(title) # 设置图的 title
    plt.xlabel('Training examples') # 横坐标
    plt.ylabel('Score') # 纵坐标
    train_sizes, train_scores, test_scores = learning_curve(estimator, X, y, cv=cv,
                                                            train_sizes=train_sizes, scoring='f1')#画学习曲线 
    train_scores_mean = np.mean(train_scores, axis=1) # 计算平均值
    train_scores_std = np.std(train_scores, axis=1) # 计算标准差
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid() # 设置背景的网格
    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std,
                     alpha=0.1, color='g') # 设置颜色
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std,
                     alpha=0.1, color='r')
    
    plt.plot(train_sizes, train_scores_mean, 'o-', color='g',
             label='traning score') # 绘制训练精度曲线
    for a, b in zip(train_sizes, train_scores_mean):  
        plt.text(a, b, (a,b),ha='center', va='bottom', fontsize=10)
    plt.plot(train_sizes, test_scores_mean, 'o-', color='r',
             label='testing score') # 绘制测试精度曲线
    for a, b in zip(train_sizes, test_scores_mean):  
        plt.text(a, b, (a,b),ha='center', va='bottom', fontsize=10)
    plt.legend(loc='best')
    return plt

g = plot_learning_curve(AdaBoostClassifier(), 'RFC', X, y) # 调用函数 plot_learning_curve 绘制随机森林学习器学习曲线

## 随机森林

In [ ]:
from sklearn.model_selection import learning_curve
import matplotlib.pyplot as plt

# 定义函数 plot_learning_curve 绘制学习曲线。
#train_sizes 初始化为 array([ 0.1  ,  0.325,  0.55 ,  0.775,  1\.   ]),
#cv 初始化为 10，以后调用函数时不再输入这两个变量

def plot_learning_curve(estimator, title, X, y, cv=10,
                        train_sizes=np.linspace(.1, 1.0, 5)):
    plt.figure(figsize=(20,10))
    plt.title(title) # 设置图的 title
    plt.xlabel('Training examples') # 横坐标
    plt.ylabel('Score') # 纵坐标
    train_sizes, train_scores, test_scores = learning_curve(estimator, X, y, cv=cv,
                                                            train_sizes=train_sizes, scoring='f1')#画学习曲线 
    train_scores_mean = np.mean(train_scores, axis=1) # 计算平均值
    train_scores_std = np.std(train_scores, axis=1) # 计算标准差
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid() # 设置背景的网格
    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std,
                     alpha=0.1, color='g') # 设置颜色
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std,
                     alpha=0.1, color='r')
    
    plt.plot(train_sizes, train_scores_mean, 'o-', color='g',
             label='traning score') # 绘制训练精度曲线
    for a, b in zip(train_sizes, train_scores_mean):  
        plt.text(a, b, (a,b),ha='center', va='bottom', fontsize=10)
    plt.plot(train_sizes, test_scores_mean, 'o-', color='r',
             label='testing score') # 绘制测试精度曲线
    for a, b in zip(train_sizes, test_scores_mean):  
        plt.text(a, b, (a,b),ha='center', va='bottom', fontsize=10)
    plt.legend(loc='best')
    return plt
g = plot_learning_curve(RandomForestClassifier(), 'RFC', X, y) # 调用函数 plot_learning_curve 绘制随机森林学习器学习曲线

In [ ]:
feature.to_csv('feature.csv')

# 决策树 集成学习 随机森林算法 参数调节

###### Python sklearn 参数调节

![image](https://doc.shiyanlou.com/document-uid598017labid4262timestamp1513834446518.png)

| 参数              |                                                        | 特点                                           |*
| ----------------- | ------------------------------------------------------ | ---------------------------------------------- |
| n_estimators      | 基学习器数目（默认值 10）                              | 基本趋势是值越大精度越高 ，直到达到一个上限    |
| criterion         | 选择算法 gini 或者 entropy (默认 gini)                 | 视具体情况定                                   |
| max_features      | 2.2.3 节中子集的大小，即 k 值（默认 sqrt(n_features)） |                                                |
| max_depth         | 决策树深度                                             | 过小基学习器欠拟合，过大基学习器过拟合。粗调节 |
| max_leaf_nodes    | 最大叶节点数（默认无限制）                             | 粗调节                                         |
| min_samples_split | 分裂时最小样本数，默认 2                               | 细调节, 越小模型越复杂                         |
| min_samples_leaf  | 叶节点最小样本数，默认 2                               | 细调节，越小模型越复杂                         |
| bootstrap         | 是否采用自助法进行样本抽样（默认使用）                 | 决定基学习器样本是否一致                       |

在以上参数中，只有 n_estimators 对精度的影响是单调的。粗调节表示参数选择跨度大，以 10、100 等为单位。细调节参数选择跨度小，以 1、2 等为单位。

 #####代码实现

 #####交叉验证法调参

首先调节：`n_estimators`，`max_depth`。首先观察特征数目，这决定了 `max_depth` 等参数的范围。然后使用交叉验证法调参。

得到最优参数 `n_estimators=60，max_depth=8`。

n_estimators:  
也就是弱学习器的最大迭代次数，或者说最大的弱学习器的个数。  
一般来说n_estimators太小，容易过拟合，n_estimators太大，又容易欠拟合，一般选择一个适中的数值。默认是100。  
在实际调参的过程中，我们常常将n_estimators和下面介绍的参数learning_rate一起考虑。  
max_depth  
决策树最大深度max_depth, 默认可以不输入，如果不输入的话，决策树在建立子树的时候不会限制子树的深度。  
一般来说，数据少或者特征少的时候可以不管这个值。如果模型样本量多，特征也多的情况下  
，推荐限制这个最大深度，具体的取值取决于数据的分布。常用的可以取值10-100之间。  
参数效果：值越大，决策树越复杂，越容易过拟合。  

## 交叉验证法调参
首先调节：n_estimators，max_depth。首先观察特征数目，这决定了 max_depth 等参数的范围。然后使用交叉验证法调参。

得到最优参数 n_estimators=60，max_depth=8。

### DesitionTreeeClassifier

In [ ]:
DecisionTreeClassifier()

In [ ]:
def para_tune(para, X, y): #
    clf = DecisionTreeClassifier(max_depth=para) # n_estimators 设置为 para
    score = np.mean(cross_val_score(clf, X, y, scoring='f1'))
    return score

def accurate_curve(para_range, X, y, title):
    score = []
    for para in para_range:
        score.append(para_tune(para, X, y))
    plt.figure(figsize=(10,5))
    plt.title(title)
    plt.xlabel('Paramters')
    plt.ylabel('Score')
    plt.grid()
    plt.plot(para_range, score, 'o-')
    for a, b in zip(para_range,  score):  
        plt.text(a, b, (a,b),ha='center', va='bottom', fontsize=10)
    return plt

g = accurate_curve([1,2,3,4,5,6,7], X, y, 'n_estimator tuning')

In [ ]:
def para_tune(para, X, y): #
    clf = DecisionTreeClassifier(criterion=para) # n_estimators 设置为 para
    score = np.mean(cross_val_score(clf, X, y, scoring='f1'))
    return score

def accurate_curve(para_range, X, y, title):
    score = []
    for para in para_range:
        score.append(para_tune(para, X, y))
    plt.figure(figsize=(10,5))
    plt.title(title)
    plt.xlabel('Paramters')
    plt.ylabel('Score')
    plt.grid()
    plt.plot(para_range, score, 'o-')
    for a, b in zip(para_range,  score):  
        plt.text(a, b, (a,b),ha='center', va='bottom', fontsize=10)
    return plt

g = accurate_curve(['entropy','gini'], X, y, 'criterion tuning')

### RandomForestClassifier

In [ ]:
def para_tune(para, X, y): #
    clf = RandomForestClassifier(n_estimators=para) # n_estimators 设置为 para
    score = np.mean(cross_val_score(clf, X, y, scoring='f1'))
    return score

def accurate_curve(para_range, X, y, title):
    score = []
    for para in para_range:
        score.append(para_tune(para, X, y))
    plt.figure(figsize=(10,5))
    plt.title(title)
    plt.xlabel('Paramters')
    plt.ylabel('Score')
    plt.grid()
    plt.plot(para_range, score, 'o-')
    for a, b in zip(para_range,  score):  
        plt.text(a, b, (a,b),ha='center', va='bottom', fontsize=10)
    return plt

g = accurate_curve([2, 10,20,30,40, 50,60, 80], X, y, 'n_estimator tuning')

n_estimators 最优50

In [ ]:
def para_tune(para, X, y):
    clf = RandomForestClassifier(n_estimators=50, max_depth=para)
    score = np.mean(cross_val_score(clf, X, y, scoring='f1'))
    return score

def accurate_curve(para_range, X, y, title):
    score = []
    for para in para_range:
        score.append(para_tune(para, X, y))
    plt.figure(figsize=(10,5))
    plt.title(title)
    plt.xlabel('Paramters')
    plt.ylabel('Score')
    plt.grid()
    plt.plot(para_range, score, 'o-')
    for a, b in zip(para_range,  score):  
        plt.text(a, b, (a,b),ha='center', va='bottom', fontsize=10)
    return plt

g = accurate_curve([2,4,6, 8,10, 12], X, y, 'max_depth tuning')

max_depth=8

In [ ]:
# para_grid = {'max_depth': [6], 'n_estimators': [50], 'max_features': [1, 5, 10], 'criterion': ['gini', 'entropy'],
#              'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 5, 10]}#对以上参数进行网格搜索


In [ ]:
def para_tune(para, X, y):
    clf = RandomForestClassifier(n_estimators=50,max_depth=8, max_features=para)
    score = np.mean(cross_val_score(clf, X, y, scoring='f1'))
    return score

def accurate_curve(para_range, X, y, title):
    score = []
    for para in para_range:
        score.append(para_tune(para, X, y))
    plt.figure(figsize=(10,5))
    plt.title(title)
    plt.xlabel('Paramters')
    plt.ylabel('Score')
    plt.grid()
    plt.plot(para_range, score, 'o-')
    for a, b in zip(para_range,  score):  
        plt.text(a, b, (a,b),ha='center', va='bottom', fontsize=10)
    return plt

g = accurate_curve([1,2,4,6, 8,10, 12], X, y, 'max_features tuning')

max_features=10

In [ ]:
def para_tune(para, X, y):
    clf = RandomForestClassifier(n_estimators=50,max_depth=8,min_samples_split=para, max_features=10)
    score = np.mean(cross_val_score(clf, X, y, scoring='f1'))
    return score

def accurate_curve(para_range, X, y, title):
    score = []
    for para in para_range:
        score.append(para_tune(para, X, y))
    plt.figure(figsize=(10,5))
    plt.title(title)
    plt.xlabel('Paramters')
    plt.ylabel('Score')
    plt.grid()
    plt.plot(para_range, score, 'o-')
    for a, b in zip(para_range,  score):  
        plt.text(a, b, (a,b),ha='center', va='bottom', fontsize=10)
    return plt

g = accurate_curve([2,4,6, 8,10, 12], X, y, 'min_samples_split tuning')

max_features=6

### AdaboostClassifier

## scikit-learn 自动调参函数 GridSearchCV
接下来使用这个函数来选择最优的学习器，并绘制上一节实验学到的学习曲线。

观察学习曲线，训练精度随样例数目增加而减小，测试精度则增加，过拟合程度降低。并且从学习曲线的变化趋势看，测试精度将随着训练样例的数目的增加而进一步增加。实际上，决策树的深度以及基学习器的数目起主要的作用。

| 参数              |                                                        | 特点                                           |*
| ----------------- | ------------------------------------------------------ | ---------------------------------------------- |
| n_estimators      | 基学习器数目（默认值 10）                              | 基本趋势是值越大精度越高 ，直到达到一个上限    |
| criterion         | 选择算法 gini 或者 entropy (默认 gini)                 | 视具体情况定                                   |
| max_features      | 2.2.3 节中子集的大小，即 k 值（默认 sqrt(n_features)） |                                                |
| max_depth         | 决策树深度                                             | 过小基学习器欠拟合，过大基学习器过拟合。粗调节 |
| max_leaf_nodes    | 最大叶节点数（默认无限制）                             | 粗调节                                         |
| min_samples_split | 分裂时最小样本数，默认 2                               | 细调节, 越小模型越复杂                         |
| min_samples_leaf  | 叶节点最小样本数，默认 2                               | 细调节，越小模型越复杂                         |
| bootstrap         | 是否采用自助法进行样本抽样（默认使用）                 | 决定基学习器样本是否一致                       |

在以上参数中，只有 n_estimators 对精度的影响是单调的。粗调节表示参数选择跨度大，以 10、100 等为单位。细调节参数选择跨度小，以 1、2 等为单位。

### RandomForestClassifier

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import learning_curve

def plot_learning_curve(estimator, title, X, y, cv=10,
                        train_sizes=np.linspace(.1, 1.0, 5)):
    plt.figure()
    plt.title(title) # 设置图的 title
    plt.xlabel('Training examples') # 横坐标
    plt.ylabel('Score') # 纵坐标
    train_sizes, train_scores, test_scores = learning_curve(estimator, X, y, cv=cv,
                                                            train_sizes=train_sizes) 
    train_scores_mean = np.mean(train_scores, axis=1) # 计算平均值
    train_scores_std = np.std(train_scores, axis=1) # 计算标准差
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid() # 设置背景的网格
    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std,
                     alpha=0.1, color='g') # 设置颜色
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std,
                     alpha=0.1, color='r')
    plt.plot(train_sizes, train_scores_mean, 'o-', color='g',
             label='traning score') # 绘制训练精度曲线
    plt.plot(train_sizes, test_scores_mean, 'o-', color='r',
             label='testing score') # 绘制测试精度曲线
    plt.legend(loc='best')
    return plt

clf = RandomForestClassifier()
para_grid = {'max_depth': [4,5,6,7], 'n_estimators': [50,60,70], 'max_features': [12,15,18], 'criterion': ['gini', 'entropy'],
             'min_samples_split': [2,3,4], 'min_samples_leaf': [2,3,4]}#对以上参数进行网格搜索
gs = GridSearchCV(clf, param_grid=para_grid, cv=3, scoring='f1')
gs.fit(X, y)
gs_best = gs.best_estimator_ #选择出最优的学习器
print(gs.best_score_) #最优学习器的精度
print(gs_best)

g = plot_learning_curve(gs_best, 'RFC', X, y)#调用实验2中定义的 plot_learning_curve 绘制学习曲线

### DesitionTreeClassifier

In [ ]:
  1.criterion  gini  or  entropy
  2.splitter  best or random 前者是在所有特征中找最好的切分点 后者是在部分特征中（数据量大的时候）
  3.max_features  默认是None（所有），log2，sqrt，N  特征小于50的时候一般使用所有的
  #N就是特征属性的个数
  4.max_depth  数据少或者特征少的时候可以不管这个值，如果模型样本量多，特征也多的情况下，可以尝试限制下
  #防止过拟合
  5.min_samples_split  如果某节点的样本数少于min_samples_split，则不会继续再尝试选择最优特征来进行划分。
                       如果样本量不大，不需要管这个值。如果样本量数量级非常大，则推荐增大这个值。
  6.min_samples_leaf  这个值限制了叶子节点最少的样本数，如果某叶子节点数目小于样本数，则会和兄弟节点一起被
                      剪枝，如果样本量不大，不需要管这个值，大些如10W可是尝试下5
  7.min_weight_fraction_leaf 这个值限制了叶子节点所有样本权重和的最小值，如果小于这个值，则会和兄弟节点一起
                          被剪枝默认是0，就是不考虑权重问题。一般来说，如果我们有较多样本有缺失值，
                          或者分类树样本的分布类别偏差很大，就会引入样本权重，这时我们就要注意这个值了。
  8.max_leaf_nodes 通过限制最大叶子节点数，可以防止过拟合，默认是"None”，即不限制最大的叶子节点数。
                   如果加了限制，算法会建立在最大叶子节点数内最优的决策树。
                   如果特征不多，可以不考虑这个值，但是如果特征分成多的话，可以加以限制
                   具体的值可以通过交叉验证得到。
  9.class_weight 指定样本各类别的的权重，主要是为了防止训练集某些类别的样本过多
                 导致训练的决策树过于偏向这些类别。这里可以自己指定各个样本的权重
                 如果使用“balanced”，则算法会自己计算权重，样本量少的类别所对应的样本权重会高。
  10.min_impurity_split 这个值限制了决策树的增长，如果某节点的不纯度
                       (基尼系数，信息增益，均方差，绝对差)小于这个阈值
                       则该节点不再生成子节点。即为叶子节点 。

In [ ]:
## 输出学习器

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import learning_curve

def plot_learning_curve(estimator, title, X, y, cv=10,
                        train_sizes=np.linspace(.1, 1.0, 5)):
    plt.figure()
    plt.title(title) # 设置图的 title
    plt.xlabel('Training examples') # 横坐标
    plt.ylabel('Score') # 纵坐标
    train_sizes, train_scores, test_scores = learning_curve(estimator, X, y, cv=cv,
                                                            train_sizes=train_sizes) 
    train_scores_mean = np.mean(train_scores, axis=1) # 计算平均值
    train_scores_std = np.std(train_scores, axis=1) # 计算标准差
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid() # 设置背景的网格
    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std,
                     alpha=0.1, color='g') # 设置颜色
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std,
                     alpha=0.1, color='r')
    plt.plot(train_sizes, train_scores_mean, 'o-', color='g',
             label='traning score') # 绘制训练精度曲线
    plt.plot(train_sizes, test_scores_mean, 'o-', color='r',
             label='testing score') # 绘制测试精度曲线
    plt.legend(loc='best')
    return plt

clf = DecisionTreeClassifier()
para_grid = {'max_depth': [3,4,5], 'max_features': [5,8,10,12,14],'criterion': ['gini', 'entropy'],
             'min_samples_split': [5,12,14,16,18,20] ,'min_samples_leaf': [1,2,3,4]}#对以上参数进行网格搜索
gs = GridSearchCV(clf, param_grid=para_grid, cv=3, scoring='f1')
gs.fit(X, y)
gs_best = gs.best_estimator_ #选择出最优的学习器
print(gs.best_score_) #最优学习器的精度
print(gs_best)
g = plot_learning_curve(gs_best, 'RFC', X, y)#调用实验2中定义的 plot_learning_curve 绘制学习曲线

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=4,
            max_features=10, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=3, min_samples_split=5,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')
            
DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
            max_features=10, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=3, min_samples_split=12,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

DecisionTree的
dict_keys(['cv', 'error_score', 'estimator__class_weight', 'estimator__criterion', 'estimator__max_depth', 'estimator__max_features', 'estimator__max_leaf_nodes', 'estimator__min_impurity_decrease', 'estimator__min_impurity_split', 'estimator__min_samples_leaf', 'estimator__min_samples_split', 'estimator__min_weight_fraction_leaf', 'estimator__presort', 'estimator__random_state', 'estimator__splitter', 'estimator', 'fit_params', 'iid', 'n_jobs', 'param_grid', 'pre_dispatch', 'refit', 'return_train_score', 'scoring', 'verbose'])


In [ ]:
gs.get_params().keys()
{'estimator__max_depth': [6], 'estimator__max_features': [1, 5, 10], 'estimator__criterion': ['gini', 'entropy'],
             'estimator__min_samples_split': [2, 5, 10], 'estimator__min_samples_leaf': [1, 5, 10]}

### AdaBoostClassifier

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import learning_curve
from sklearn.tree import DecisionTreeClassifier
def plot_learning_curve(estimator, title, X, y, cv=10,
                        train_sizes=np.linspace(.1, 1.0, 5)):
    plt.figure()
    plt.title(title) # 设置图的 title
    plt.xlabel('Training examples') # 横坐标
    plt.ylabel('Score') # 纵坐标
    train_sizes, train_scores, test_scores = learning_curve(estimator, X, y, cv=cv,
                                                            train_sizes=train_sizes) 
    train_scores_mean = np.mean(train_scores, axis=1) # 计算平均值
    train_scores_std = np.std(train_scores, axis=1) # 计算标准差
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid() # 设置背景的网格

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std,
                     alpha=0.1, color='g') # 设置颜色
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std,
                      alpha=0.1, color='r')
    plt.plot(train_sizes, train_scores_mean, 'o-', color='g',
             label='traning score') # 绘制训练精度曲线
    plt.plot(train_sizes, test_scores_mean, 'o-', color='r',
             label='testing score') # 绘制测试精度曲线
    plt.legend(loc='best')
    return plt

# param_grid = {dtc__criterion : ["gini", "entropy"],
#               dtc__splitter :   ["best", "random"],
#               abc__n_estimators: [none, 1, 2]
#              }
# DTC = DecisionTreeClassifier(random_state = 11, max_features = "auto", class_weight = "auto",max_depth = None)
# ABC = AdaBoostClassifier(base_estimator = DTC)
# # run grid search
# grid_search_ABC = GridSearchCV(ABC, param_grid=param_grid, scoring = 'roc_auc')


clf = AdaBoostClassifier(base_estimator=DecisionTreeClassifier())
# para_grid = {'max_depth': [10], 'n_estimators': [100], 'max_features': [1, 5, 10], 'criterion': ['gini', 'entropy'],
#              'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 5, 10]}#对以上参数进行网格搜索
para_grid ={"base_estimator__criterion": ["gini", "entropy"],
          "base_estimator__splitter":   [ "random"],
          "n_estimators": [1],'base_estimator__max_depth': [3,5,7], 'base_estimator__max_features': [12,15,17,19],
             'base_estimator__min_samples_split': [7,10,12,15], 'base_estimator__min_samples_leaf': [1,2,3,5,7]}
# class_weight=None, criterion='gini', max_depth=None,
#             max_features=None, max_leaf_nodes=None,
#             min_impurity_decrease=0.0, min_impurity_split=None,
#             min_samples_leaf=1, min_samples_split=2,
#             min_weight_fraction_leaf=0.0, presort=False, random_state=None,
#             splitter='best'),
#           learning_rate=1.0, n_estimators=50, random_state=None
gs = GridSearchCV(clf, param_grid=para_grid, cv=3, scoring='f1')
gs.fit(X, y)
gs_best = gs.best_estimator_ #选择出最优的学习器
print(gs.best_score_) #最优学习器的精度
print(gs_best)

g = plot_learning_curve(gs_best, 'RFC', X, y)#调用实验2中定义的 plot_learning_curve 绘制学习曲线

In [ ]:
gs.get_params().keys()

In [ ]:

gs = GridSearchCV(DecisionTreeClassifier(), param_grid=para_grid, cv=3, scoring='f1')
gs.get_params().keys()

### XGBoost

In [ ]:
from xgboost import XGBClassifier
clf1 = XGBClassifier(max_depth=6,n_estimators=200)#.fit(train_data,train_label)
score1 =cross_val_score(clf1,X,y,cv=5,scoring='f1')
print(np.mean( score1))

clf2 = XGBClassifier(max_depth=4,n_estimators=292)#.fit(train_data,train_label)
score2 = cross_val_score(clf2,X,y,cv=5,scoring='f1')
print (np.mean(score2))

In [ ]:
from xgboost import XGBClassifier

In [ ]:
XGBClassifier()

XGB需要调整的参数  

max_depth = 5 :  
和GBM中的参数相同，这个值为树的最大深度。  
这个值也是用来避免过拟合的。max_depth越大，模型会学到更具体更局部的样本。  
需要使用CV函数来进行调优。  
典型值：3-10  
min_child_weight = 1:  
决定最小叶子节点样本权重和。  
和GBM的 min_child_leaf 参数类似，但不完全一样。XGBoost的这个参数是最小样本权重的和，而GBM参数是最小样本总数。  
这个参数用于避免过拟合。当它的值较大时，可以避免模型学习到局部的特殊样本。  
但是如果这个值过高，会导致欠拟合。这个参数需要使用CV来调整。  
gamma = 0:  
在节点分裂时，只有分裂后损失函数的值下降了，才会分裂这个节点。Gamma指定了节点分裂所需的最小损失函数下降值。  
这个参数的值越大，算法越保守。这个参数的值和损失函数息息相关，所以是需要调整的。  
subsample：  
和GBM中的subsample参数一模一样。这个参数控制对于每棵树，随机采样的比例。  
减小这个参数的值，算法会更加保守，避免过拟合。但是，如果这个值设置得过小，它可能会导致欠拟合。  
典型值：0.5-1  
 colsample_bytree：  
和GBM里面的max_features参数类似。用来控制每棵随机采样的列数的占比(每一列是一个特征)。  
典型值：0.5-  
scale_pos_weight = 1:  
在各类别样本十分不平衡时，把这个参数设定为一个正值，可以使算法更快收敛。  
以上是我们要进行调参的部分来优化结果（当然有时候n_estimators(迭代次数）也能起到优化作用）  
一下过程我们用一步步修改的方法，来查看结果，用for 函数来列举各个调参过程，for函数我就不列举了，直接通过for得出的结果给大家列举最有参数。当然你也可以不用for 来做，可以用sklearn.moedel_selection的  GridSearchCV来快速调参。  

我们先从n_estimators  来定  

'n_estimators':[100,200,500,1000,1500]  

n_estimators:[500] (500,600,700,800)

min_child_weight:[2] (1,3,5)

max_depth:[6] (3-10)

gamma:[0]

subsample:[1] (0.5-1)

colsample_bytree:[0.7] (0.5-1)

reg_alpha:[0] (0.01,0.5,1)

reg_lambda:[1] （0.01-0.1，1）

‍learning_rate:[0.05] (0.01-0.3)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import learning_curve
from xgboost import XGBClassifier
import warnings
warnings.filterwarnings("ignore")
def plot_learning_curve(estimator, title, X, y, cv=10,
                        train_sizes=np.linspace(.1, 1.0, 5)):
    plt.figure()
    plt.title(title) # 设置图的 title
    plt.xlabel('Training examples') # 横坐标
    plt.ylabel('Score') # 纵坐标
    train_sizes, train_scores, test_scores = learning_curve(estimator, X, y, cv=cv,
                                                            train_sizes=train_sizes) 
    train_scores_mean = np.mean(train_scores, axis=1) # 计算平均值
    train_scores_std = np.std(train_scores, axis=1) # 计算标准差
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid() # 设置背景的网格

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std,
                     alpha=0.1, color='g') # 设置颜色
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std,
                      alpha=0.1, color='r')
    plt.plot(train_sizes, train_scores_mean, 'o-', color='g',
             label='traning score') # 绘制训练精度曲线
    plt.plot(train_sizes, test_scores_mean, 'o-', color='r',
             label='testing score') # 绘制测试精度曲线
    plt.legend(loc='best')
    return plt


clf = XGBClassifier()
# XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
#        colsample_bynode=1, colsample_bytree=1, gamma=0, learning_rate=0.1,
#        max_delta_step=0, max_depth=3, min_child_weight=1, missing=None,
#        n_estimators=100, n_jobs=1, nthread=None,
#        objective='binary:logistic', random_state=0, reg_alpha=0,
#        reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
#        subsample=1, verbosity=1)
# 'n_estimators':[100,200,500,1000,1500]
# n_estimators:[500] (500,600,700,800)
# min_child_weight:[2] (1,3,5)
# max_depth:[6] (3-10)
# gamma:[0]
# subsample:[1] (0.5-1)
# colsample_bytree:[0.7] (0.5-1)
# reg_alpha:[0] (0.01,0.5,1)
# reg_lambda:[1] （0.01-0.1，1）
# ‍learning_rate:[0.05] (0.01-0.3)
# para_grid ={"n_estimators":[250,300,350],"max_depth":[2,3,4],'colsample_bytree':[0.5,0.6,0.7]}
para_grid ={"n_estimators":[150,200,250],"max_depth":[2,3,4],'colsample_bytree':[0.3,0.4,0.5]}
gs = GridSearchCV(clf, param_grid=para_grid, cv=3, scoring='f1')
gs.fit(X, y)
gs_best = gs.best_estimator_ #选择出最优的学习器
print(gs.best_score_) #最优学习器的精度
print(gs_best)

g = plot_learning_curve(gs_best, 'RFC', X, y)#调用实验2中定义的 plot_learning_curve 绘制学习曲线

In [ ]:
gs_best

## 其他方式交叉验证的过程
#### cv=None 交叉验证参数，默认None，使用三折交叉验证。指定fold数量，默认为3，也可以是yield训练/测试数据的生成器。

### GradientBoostingClassifier

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
clf1 = GradientBoostingClassifier(max_depth=3,n_estimators=289)#.fit(train_data,train_label) 

np.mean(cross_val_score(clf1,X,y,cv=5,scoring='f1'))

#### 解释为什么所有地方都用f1而不用precision

常用的是 ‘precision’ 和 ’recall‘ 和 ’f1，三者的关系可以用下图来表示

 ![img](https://img-blog.csdn.net/20180924203640831?watermark/2/text/aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3FxXzMyNTkwNjMx/font/5a6L5L2T/fontsize/400/fill/I0JBQkFCMA==/dissolve/70) 

假设这是一个二元分类的问题

准确率（precision）也就是被分类器检测到的数据中 分类正确的部分

召回率（recall）就是 正类中被分类正确的部分

而F1值就是 准确率和召回率的调和平均数

在实际应用中，如果是做搜索类的问题，那就是在保证召回率的情况下提升准确率

在做垃圾邮件检测之类的问题，就是要保证准确率的情况下提升召回率

具体也就是遇到具体问题看两者的权衡

如果两者都要求高，那就需要保证较高的F1 score


机器学习中的 precision、recall、accuracy、F1 Score
1. 四个概念定义：TP、FP、TN、FN
先看四个概念定义： 
- TP，True Positive 
- FP，False Positive 
- TN，True Negative 
- FN，False Negative

如何理解记忆这四个概念定义呢？

举个简单的二元分类问题 例子：

假设，我们要对某一封邮件做出一个判定，判定这封邮件是垃圾邮件、还是这封邮件不是垃圾邮件？

如果判定是垃圾邮件，那就是做出（Positive）的判定； 
如果判定不是垃圾邮件，那就做出（Negative）的判定。

True Positive（TP）意思表示做出Positive的判定，而且判定是正确的。因此，TP的数值表示正确的Positive判定的个数。 
同理，False Positive（TP）数值表示错误的Positive判定的个数。 
依此，True Negative（TN）数值表示正确的Negative判定个数。 
False Negative（FN）数值表示错误的Negative判定个数。

2. Precision、Recall、Accuracy、F1 Score（F Score）
四个概念定义：

precision = TP / (TP + FP)
recall = TP / (TP + FN)
accuracy = (TP + TN) / (TP + FP + TN + FN)
F1 Score = P*R/2(P+R)，其中P和R分别为 precision 和 recall
如果某个二元分类问题，训练拟合得到了几个模型假设，那么通常我们选择在验证集上，F1 Score 数值最大的那个模型假设。

### XGBClassifier

In [ ]:
from xgboost import XGBClassifier
clf1 = XGBClassifier(max_depth=6,n_estimators=200)#.fit(train_data,train_label)
score1 =cross_val_score(clf1,X,y,cv=5,scoring='f1')
print(np.mean( score1))

clf2 = XGBClassifier(max_depth=4,n_estimators=292)#.fit(train_data,train_label)
score2 = cross_val_score(clf2,X,y,cv=5,scoring='f1')
print (np.mean(score2))

### SVC

In [ ]:
from sklearn.svm import SVC
# from sklearn.datasets import make_blobs

# X, y = make_blobs(n_samples=100, centers=2, random_state=0, cluster_std=0.3)

clf = SVC(C = 1.0, kernel='linear')
clf.fit(X,y)

print(clf.score(X,y))
plt.figure(figsize=(10,3), dpi=100)
plot_hyperplane(clf, X, y, h=0.01, title='Maximiin Margin Hyperplan')

In [ ]:
from sklearn.svm import SVC
clf1 = SVC(kernel='rbf', C=16, gamma=0.18)#.fit(train_data,train_label)
score1 = cross_val_score(clf1,X,y,cv=5,scoring='f1')
print(np.mean( score1))

clf2 = SVC(kernel='rbf', C=94.75, gamma=0.17)#.fit(train_data,train_label)
score3 = cross_val_score(clf2,X,y,cv=5,scoring='f1')
print (np.mean(score2))

### 再次RandomForest

In [ ]:
score2 = cross_val_score(clf,X,y,cv=5,scoring='f1')
print (np.mean(score2))

# 最终预测

In [ ]:
clf.predict(X)

In [ ]:
print("The model is trained on the full development set.")
print("The scores are computed on the full evaluation set.")
print()
#在测试集上测试最优的模型的泛化能力.
y_true, y_pred = y, clf.predict(X)
print(classification_report(y_true, y_pred))

In [ ]:
#用sklearn的SVC方法来训练数据集，并交叉验证预测精度  
from sklearn import metrics  
# import matplotlib.pyplot as plt  
# import pandas as pd  
  
# tbl = pd.read_csv("./data/bmi.csv")  
# #读取数据  
  
# label = tbl["label"]  
# #读取数据中的标签列  
# w = tbl["weight"] / 100   
# h = tbl["height"] / 200   
# wh = pd.concat([w, h], axis=1)  
  
# data_train, data_test, label_train, label_test = cross_validation.train_test_split(wh, label)  
# #将数据分成两组数据集和测试集  
print(clf)
# # clf = svm.SVC()  
# clf.fit(X,y)  
# #训练数据  
  
predict = clf.predict(X)  
#预测数据  
print(str(predict.tolist()).count("0"))
print(str(predict.tolist()).count("1"))
ac_score = metrics.accuracy_score(y, predict)  
# #生成测试精度  
cl_report = metrics.classification_report(y, predict)  
# #生成交叉验证的报告  
print(ac_score)  
# #显示数据精度  
print(cl_report)  
# #显示交叉验证数据集报告  

In [ ]:
from scipy import integrate
import numpy as np

def f(x):
    return (pow(x,3)*np.cos(x/2)+1/2)*np.sqrt(4-pow(x,2))
x = np.linspace(-2,2,1000)
y = f(x)
v = integrate.trapz(y, x)
print(v)
